In [ ]:
!pip install langchain openai pinecone-client langchain-community sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 556.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# 벡터 DB 및 llm 라이브러리
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import openai

# 임베딩 모델 위한 라이브러리
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd

In [ ]:
# 1. OpenAI API 키 설정 및 임베딩 모델 초기화
openai_api_key = "your-api-key"

In [ ]:
# multi-qa-mpnet-base-dot-v1 모델 로드
#model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 2. 파인콘 초기화
from pinecone import Pinecone, ServerlessSpec

pinecone = Pinecone(api_key="your-api-key")
index = pinecone.Index("trip-index")

In [ ]:
# 3. 검색 함수 정의

# 선호 숙소 형태 및 동행인 기반
def search_places(city, companions, lodging_style):
    query = f"Best accommdations in {city} for {companions} with focus on {lodging_style}."
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_lodging"
    results_style = index.query(vector=query_embedding, top_k=20, namespace=namespace, include_metadata=True)
    return results_style

In [ ]:
# 4. 숙소 리스트 생성을 위한 프롬프트 템플릿 - 영어 버전
prompt_template = prompt = """
Imagine you're a travel agent to recommend an accommodation list to your customers.
Your task is to recommend an accommodation list to your customers based on their <personal information> and <itinerary>, as written below.
Based on the <requirements> below, create a list of the **top 5** accommodations for your customer's trip.

<personal infomation>
- Travel City: {city}
- Duration of Stay: {trip_duration}
- Travel Dates: {travel_dates}
- Companions: {companions}
- Accommodation Preferences: {lodging_style}
- Accommodation budget limit per Night: {lodging_budget}
</personal infomation>

<itinerary>
- itinerary: {itinerary}
</itinerary>

<requirements>
- You should consider their accommodation preferences and accommodation budget per night.
- Provide a concise description of each accommodation’s features.
- Include location details such as proximity to major attractions and transportation convenience.
- Ensure the accommodations are in safe areas of {city}.
- Offer the proper option based on their travel companions among hotels, resorts, inns, hostels, B&B, and so on.
- Please make the list consist of the {recommendations}.
- **Only for an approximate nightly price during {travel_dates}, please retrieve this information from external sources.**
- Ensure the budget per night is not exceeded. The budget currency is the Korean won(KRW).
- **Please answer in Korean.**
- Provide the results in JSON format following the <output structure> below.
</requirements>

<output structure>
Please include location, features, services, rating, and 'approximate' price(KRW) in the output.
For the price, specify the currency.
</output structure>
"""

In [ ]:
# 5. 프롬프트와 LLMChain 설정
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o",    # 4-turbo보다 빠르고, 한국어도 더 잘함
    openai_api_key = openai_api_key
)

# 숙소 추천 생성 함수
def generate_accommodation_recommendations(city, trip_duration, travel_dates, companions, lodging_style, lodging_budget, recommendations, itinerary):
    # 템플릿에 사용자 정보 삽입
    formatted_prompt = prompt_template.format(
        city=city,
        trip_duration=trip_duration,
        travel_dates=travel_dates,
        companions=companions,
        lodging_style=lodging_style,
        lodging_budget=lodging_budget,
        recommendations=recommendations,
        itinerary=itinerary
    )

    # LangChain 프롬프트 구성
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template("You are an AI expert in accommodation recommendations."),
            HumanMessagePromptTemplate.from_template(formatted_prompt)
        ]
    )

    # LLMChain 설정 및 실행
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True
    )

    # 결과 생성
    result = conversation.run({})
    return result


In [ ]:
# 6. 메인 함수: 사용자 입력 및 숙소 추천 실행
def final_recommendations():
    # 사용자 입력 예시
    accommodation_details = {
        "city": "paris",
        "trip_duration": "three nights four days",
        "travel_dates": "2024-11-15 ~ 2024-11-18",
        "companions": "friends",
        "lodging_style": "cost-effective, clean, luggage storage",
        "lodging_budget": "100,000 won(KRW) per one night"
    }

    # 파인콘에서 숙소 검색 실행 (상위 5개 숙소)
    search_results = search_places(
        city=accommodation_details["city"],
        companions=accommodation_details["companions"],
        lodging_style=accommodation_details["lodging_style"]
    )

    # 파인콘에서 가져온 추천 숙소 리스트 구성 (최대 20개)
    recommendations = "\n".join([
        f"- {match.metadata['1_이름']} (평점: {match.metadata['3_평점']}, 위치: {match.metadata['2_주소']}, 유형: {match.metadata['8_유형']})"
        for match in search_results.matches
    ])

    # 여행일정 리스트 구성
    file_path = 'paris_itinerary.csv'   # CSV 파일 경로 설정
    paris_travel_schedule = pd.read_csv(file_path)     # CSV 파일 로드
    itinerary = "\n".join([
        f"- {row['활동']} (운영시간: {row['운영시간']}, 위치: {row['주소']})"
        for _, row in paris_travel_schedule.iterrows()
    ])

    # 숙소 추천 생성 호출
    accommodation_recommendations = generate_accommodation_recommendations(
        city=accommodation_details["city"],
        trip_duration=accommodation_details["trip_duration"],
        travel_dates=accommodation_details["travel_dates"],
        companions=accommodation_details["companions"],
        lodging_style=accommodation_details["lodging_style"],
        lodging_budget=accommodation_details["lodging_budget"],
        recommendations=recommendations,
        itinerary=itinerary
    )

    # 결과 출력
    return accommodation_recommendations

In [ ]:
# 7. 코드 실행
accommodations = final_recommendations()



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an AI expert in accommodation recommendations.
Human: 
Imagine you're a travel agent to recommend an accommodation list to your customers.
Your task is to recommend an accommodation list to your customers based on their <personal information> and <itinerary>, as written below.
Based on the <requirements> below, create a list of the **top 5** accommodations for your customer's trip.

<personal infomation>
- Travel City: paris
- Duration of Stay: three nights four days
- Travel Dates: 2024-11-15 ~ 2024-11-18
- Companions: friends
- Accommodation Preferences: cost-effective, clean, luggage storage
- Accommodation budget limit per Night: 100,000 won(KRW) per one night
</personal infomation>

<itinerary>
- itinerary: - 에펠탑 방문 (운영시간: 09:30 - 23:45, 위치: Champ de Mars, 5 Avenue Anatole France, 75007 Paris, France)
- 샹젤리제 거리 산책 및 쇼핑 (운영시간: 상점마다 다름, 위치: Avenue des Champs-Élysées, 75008 Paris, France)
- 루브르 박물관 관람 (운영시간: 

In [ ]:
print(accommodations)     # json 형태로 출력

```json
[
    {
        "name": "St Christopher's Inn Paris Gare du Nord",
        "location": "5 Rue de Dunkerque, 75010 Paris, France",
        "features": "호스텔, 저렴한 가격, 깨끗한 시설",
        "services": "수하물 보관, 무료 Wi-Fi, 공용 주방",
        "rating": 3.9,
        "approximate_price": "80,000 KRW"
    },
    {
        "name": "Hôtel Le Quartier Bercy - Square",
        "location": "33 Bd de Reuilly, 75012 Paris, France",
        "features": "호텔, 현대적인 디자인, 편안한 객실",
        "services": "수하물 보관, 무료 Wi-Fi, 24시간 리셉션",
        "rating": 4.3,
        "approximate_price": "95,000 KRW"
    },
    {
        "name": "Hôtel Bonaparte Paris",
        "location": "61 Rue Bonaparte, 75006 Paris, France",
        "features": "호텔, 역사적인 건물, 세련된 인테리어",
        "services": "수하물 보관, 무료 Wi-Fi, 조식 제공",
        "rating": 4.4,
        "approximate_price": "100,000 KRW"
    },
    {
        "name": "Hotel Residence des Arts",
        "location": "14 Rue Gît-le-Cœur, 75006 Paris, France",
        "features": "호텔, 아늑한 